## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Общая информация

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Есть пропущенные значения в стобцах 'days_employed', 'total_income'.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
mean_days_employed = df['days_employed'].mean()

In [4]:
df['days_employed'] = df['days_employed'].fillna(mean_days_employed)

В столбце 'days_employed' пропуски заменяю на среднее по столбцу, поскольку для решения задач проекта, значения данного столбца не являются критично важными

In [5]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [6]:
income_type = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'безработный', 'предприниматель', 'в декрете', 'студент']

In [7]:
for type in income_type:
    mean = df.loc[df['income_type'] == type,'total_income'].mean()
    df.loc[(df['income_type'] == type) & (df['total_income'].isnull()), 'total_income'] = mean

In [8]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Значения столбца 'total_income' понадобятся для ответа на вопрос проекта, поэтому в столбце 'total_income' пустые значения поменяла на среднее значение внутри группы по 'income_type', это самое приближенное значение, которое не сильно изменит общую картину по ежемесячным доходам

### Вывод

1)Есть пропущенные значения в стобцах 'days_employed', 'total_income'
2)Возможно, что заявитель не работает/ИП, поэтому у него нет точного стажа работы и точного ежемесячного дохода/отказался предоставлять эти данные 3)Данные заполнены по следующему принципу: по тому столбцу, значения которого не являются критично важными для решения задач проекта, заполняем пропуски - средним по столбцу. По второму столбцу, значения которого важны, заполняем пропуски - средним, рассчитанным внутри групп, выделенных категориями

### Замена типа данных

In [9]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Поменяла тип значения в столбцах 'days_employed', 'total_income' на целочисленные методом astype('int'), поскольку в данных столбцах содержатся числовые значения, количество отработанных дней и ежемесячный доход удобнее сделать целыми числами, от потери копеек и часов рез-ты не сильно изменятся

### Обработка дубликатов

In [10]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [11]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [13]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Методом unique просмотрела столбцы с типом object, для того чтобы выяснить в каких столбцах нужно привести значения к нижнему регистру

In [14]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

Привела значения в столбцах 'education' и 'family_status' к нижнему регистру, чтобы не упустить дубликаты

In [15]:
df.duplicated().sum()

71

Посчитала кол-во дубликатов. Их 71.

In [16]:
df = df.drop_duplicates().reset_index(drop=True)

Удалила дубликаты, обновили индексы

In [17]:
df.duplicated().sum()

0

Проверила, остались ли дубликаты, их не осталось

### Вывод

Дубликаты могли появиться по техническим причинам или человеческий фактор: кто-то ввел данные несколько раз. + были дубликаты из-за разных регистров, их привели к нижнему. Удалила дубликаты методом drop_duplicates. 

### Лемматизация

In [18]:
from pymystem3 import Mystem
m = Mystem()

In [19]:
def lemming(row):
        for word in row.split(' '):
            lemmas = ''.join(m.lemmatize(row))
            return lemmas

In [20]:
df['purpose_lemm'] = df['purpose'].apply(lemming)

In [21]:
df['purpose_lemm'].value_counts()

автомобиль\n                                972
свадьба\n                                   791
на проведение свадьба\n                     768
сыграть свадьба\n                           765
операция с недвижимость\n                   675
покупка коммерческий недвижимость\n         661
операция с жилье\n                          652
покупка жилье для сдача\n                   651
операция с коммерческий недвижимость\n      650
жилье\n                                     646
покупка жилье\n                             646
покупка жилье для семья\n                   638
строительство собственный недвижимость\n    635
недвижимость\n                              633
операция со свой недвижимость\n             627
строительство жилой недвижимость\n          624
покупка недвижимость\n                      621
покупка свой жилье\n                        620
строительство недвижимость\n                619
ремонт жилье\n                              607
покупка жилой недвижимость\n            

### Вывод

Для лемматизации: создала функцию, которая на аргумент принимает строку. В цикле: для каждого слова в строке(разделенного методом сплит, где разделитель "пробел"), провожу лемматизацию, склеиваю методом join.
Методом apply создаю новый столбец 'purpose_lemm' со значениями лемматизированных целей - для этого применяю созданную функцию lemming для столбца 'purpose'. Затем методом value_counts считаю количество уникальных значений в столбце 'purpose_lemm'

### Категоризация данных

In [22]:
def purpose_category(row):
    for word in row.split(' '):
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        if 'образование' in row:
            return 'образование'
        if 'недвижимость' in row or 'жилье' in row:
            return 'недвижимость'
        return 'разное'

In [23]:
df['purpose_category'] = df['purpose_lemm'].apply(purpose_category)

In [24]:
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

### Вывод

Я выделила 4 категории: автомобиль, образование, свадьба, недвижимость, так как это наиболее часто встречающиеся категории в рез-те value_counts по столбцу 'purpose_lemm', также категорию - разное, на случай, если что-то не доглядела. Написала по этим категориям условия в цикле внутри ф-ии 'purpose_category', применила получившуюся ф-ю к столбцу 'purpose_lemm' методом apply и получила новый столбец 'purpose_category', применив метод value_counts выяснила, что в таблице встречается 4 вида категории целей, самым популярным из них является кредит на недвижимость.

### Шаг 3. Выявление зависимостей

In [25]:
df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [26]:
df['gender'] = df['gender'].replace('XNA', 'M')

Далее в расчетах буду вычислять кол-во людей по столбцу 'gender'. Нашла одно уникальное значение 'XNA', предполагаю, что это ошибка, и поскольку это всего одно значение, заменяю его на М.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [27]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что есть отрицательное количество детей "-1". Всего таких значений 47 по всей таблице, это меньше 1%. Поэтому, сделав предположение, что знак "-" поставлен там случайно, и скорее всего там количество детей = "1", заменим значения в этих ячейках на "1".

In [28]:
df['children'] = df['children'].replace(-1, 1)

Также поменяем значения "20" на "2", из предположения, что "0" нажали случайно. Таких значений тоже не больше 1%, поэтому данные изменения не сильно повлияют на конечный рез-тат

In [29]:
df['children'] = df['children'].replace(20, 2)

In [30]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [31]:
df_children_debt = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
df_children_debt.columns = ['no_debt', 'debt']
df_children_debt['percent'] = df_children_debt['debt']/(df_children_debt['no_debt'] + df_children_debt['debt'])

In [32]:
df_children_debt['debt'] = df_children_debt['debt'].fillna(0)
df_children_debt['percent'] = df_children_debt['percent'].fillna(0)

In [33]:
df_children_debt.sort_values(by='percent', ascending=False)

,no_debt,debt,percent
children,,,
4,37.0,4.0,0.097561
2,1926.0,202.0,0.094925
1,4410.0,445.0,0.091658
3,303.0,27.0,0.081818
0,13028.0,1063.0,0.075438
5,9.0,0.0,0.000000


### Вывод

1)Больше всех должников среди тех, у кого 4-ро детей, однако, нельзя однозначно сказать, что чем больше детей, тем больше должников. Среди тех у кого по 2-е детей должников больше, чем среди тех, у кого 3-е. 2) Среди тех, у кого 5 детей, нет должников вообще - здесь можно предположить, что такой категории реже одобряют заявку на кредит 3) Среди тех, у кого нет детей процент должников меньше, чем у всех остальных (не считая категорию 5-детей, которую мы исключаем из предположения, что им просто не выдают кредит), из этого можно сделать вывод, что бездетные возвращают кредит в срок лучше, чем те, кто уже стал родителями.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
df_family_status_debt = df.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
df_family_status_debt.columns = ['no_debt', 'debt']
df_family_status_debt['percent'] = df_family_status_debt['debt']/(df_family_status_debt['debt'] + df_family_status_debt['no_debt'])

In [35]:
df_family_status_debt.sort_values(by = 'percent', ascending=False)

,no_debt,debt,percent
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


### Вывод

Те, кто не женат/не замужем и ни разу не был в браке, чаще опаздывают с возвратом кредита, чем все остальные. Примерно такой же уровень должников у тех, кто в гражданском браке. У женатых/замужних процент должников меньше, можно сделать вывод, что у тех, кто вступил в брак, уровень ответственности выше, чем у тех, кто не был в браке или живет гражданским браком. А те, кто в разводе или потеряли своих супругов, возвращают долги еще лучше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
df['total_income'].min()

20667

In [37]:
df['total_income'].max()

2265604

In [38]:
def income_category(income):
        if income <= 50000:
            return 'низкий'
        if income <= 90000:
            return 'ниже среднего'
        if income <= 150000:
            return 'средний'
        if income <= 300000:
            return 'выше среднего'
        return 'высокий'

In [39]:
df['income_category'] = df['total_income'].apply(income_category)

In [40]:
df_total_income_debt = df.pivot_table(index='income_category', columns='debt', values='gender', aggfunc='count')
df_total_income_debt.columns = ['no_debt', 'debt']
df_total_income_debt['percent'] = df_total_income_debt['debt']/(df_total_income_debt['debt'] + df_total_income_debt['no_debt'])

In [41]:
df_total_income_debt.sort_values(by='percent', ascending=False)

,no_debt,debt,percent
income_category,,,
средний,6587,618,0.085774
ниже среднего,2737,239,0.080309
выше среднего,8663,755,0.080166
высокий,1377,106,0.071477
низкий,349,23,0.061828


### Вывод

Можно сделать вывод, что зависимость есть. Чем выше доход - тем ниже процент должников, за исключением категории людей с низким доходом, здесь можно предположить, что это связано с тем, что такой категории реже одобряют заявку на кредит

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
df_purpose_category_debt = df.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')
df_purpose_category_debt.columns = ['no_debt', 'debt']
df_purpose_category_debt['percent'] = df_purpose_category_debt['debt']/(df_purpose_category_debt['debt'] + df_purpose_category_debt['no_debt'])

In [43]:
df_purpose_category_debt.sort_values(by='percent', ascending=False)

,no_debt,debt,percent
purpose_category,,,
автомобиль,3903,403,0.093590
образование,3643,370,0.092200
свадьба,2138,186,0.080034
недвижимость,10029,782,0.072334


### Вывод

Те, кто берет кредит на машину, чаще всего не возвращают в срок. Меньше всего должников среди тех, кто берет кредит на недвижимость.

### Шаг 4. Общий вывод

Портрет идеального заемщика - это человек без детей, в разводе или вдовец/вдова, с высоким уровнем дохода и с желанием приобрести кредит на недвижимость.